# Enrich locations and parse with GeoPandas

Learn how to enrich and prepare your address data with geospatial analytics in mind. This tutorial runs the Geocoding and Places Details APIs on a few thousand address records. We show how to preprocess tabular data as input and briefly discuss address validation. In addition, you will learn how to leverage the GeoJSON format to store results in GeoDataFrames. 

You want to repeat all the steps below on your own data? Check out how to [install a Conda virutal environment](install.md) and make sure to add `geobatchpy` and `geopandas`.

## Part 1 - data preprocessing

Our dataset for this tutorial consists of 1081 sport stadiums, mostly across Germany, Belgium, Netherlands. We generated the data using Geoapify's Places API.

In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None  # to display all columns
pd.set_option('display.expand_frame_repr', False)  # to print without line break

In [2]:
df = pd.read_csv('stadium-sample.csv', usecols=['Name', 'Street', 'Housenumber', 'Postcode', 'City', 'Country'])
print(f'Total number of records: {df.shape[0]}')
df.head()

Total number of records: 1081


,Name,Street,Housenumber,Postcode,City,Country
0,NaN,Kloaver Blatt,NaN,46342,Velen,Germany
1,"Sportanlage Königsfeld, SC Lüstringen",Hasewinkel,NaN,49086,Osnabrück,Germany
2,Ludo Coeckstadion,Roderveldlaan,NaN,2600,Antwerp,Belgium
3,NaN,Poggenbeeke,NaN,32457,Porta Westfalica,Germany
4,Standaard Muide,Galvestonstraat,NaN,9000,Ghent,Belgium


Geoapify's geocoding service accepts free text search and structured input, the latter being helpful only if we have a lot of faith in our data quality. I have seen too many data quality issues in real-world structured address records. And my conclusion is to go for the free text search. Here, we parse the structured data into one string per row.

In [3]:
def concat_columns(df: pd.DataFrame, sep: str = ' ', fill_value: str = np.nan) -> pd.Series:
    """Concatenates row-wise all columns of df and returns series with same index.
    
    """
    return (df.apply(lambda s: sep.join(s.dropna().astype(str).str.strip().replace('', np.nan).dropna()), axis=1)
            .replace('', fill_value))

df['AddressLine1'] = concat_columns(df=df[['Street', 'Housenumber']])
df['AddressLine2'] = concat_columns(df=df[['Postcode', 'City']])
addresses = concat_columns(df[['Name', 'AddressLine1', 'AddressLine2', 'Country']], sep=', ', fill_value='')
print(addresses[:5])

0                  Kloaver Blatt, 46342 Velen, Germany
1    Sportanlage Königsfeld, SC Lüstringen, Hasewin...
2    Ludo Coeckstadion, Roderveldlaan, 2600 Antwerp...
3         Poggenbeeke, 32457 Porta Westfalica, Germany
4    Standaard Muide, Galvestonstraat, 9000 Ghent, ...
dtype: object


## Part 2 - add geocodes

It is time to geocode our addresses. You can do this using our Python API but we prefer the command line. First we prepare the input file using Python:

In [4]:
from geobatchpy.batch import parse_geocoding_inputs, simplify_batch_geocoding_results
from geobatchpy.utils import write_data_to_json_file, read_data_from_json_file

In [5]:
data = {
    'api': '/v1/geocode/search',  # This tells Geoapify which service to run.
    'inputs': parse_geocoding_inputs(locations=addresses),
    'params': {'limit': 1, 'format': 'geojson'},  # We strongly recommend using GeoJSON as output format.
    'batch_len': 500,  # Results in math.ceil(1081 / 500) = 3 jobs in total.
    'id': 'tutorial-batch-geocode'
}
write_data_to_json_file(data=data, file_path='tutorial-geocode-input.json')

It is time to switch to the CLI. First we submit jobs to Geoapify servers with

```shell
geobatch submit tutorial-geocode-input.json tutorial-geocode-urls.json
```

The output of the first step `tutorial-geocode-urls.json` is the input for the next:

```shell
geobatch receive tutorial-geocode-urls.json tutorial-geocode-results.json
```

Processing on the Geoapify servers can take some time, depending on the size of your inputs, your subscription plan, and how busy the servers are. Pause here and continue after all your jobs are done.

We convert the results into a simplified list of [GeoJSON](https://geojson.org/)-like Python dictionaries. 

In [6]:
results = read_data_from_json_file('tutorial-geocode-results.json')['results']
results = simplify_batch_geocoding_results(results=results, input_format='geojson')

print('Here is how the 1st one looks like:')
results[0]

Here is how the 1st one looks like:


{'type': 'Feature',
 'properties': {'datasource': {'sourcename': 'openstreetmap',
   'attribution': '© OpenStreetMap contributors',
   'license': 'Open Database License',
   'url': 'https://www.openstreetmap.org/copyright'},
  'name': 'Kloaver Blatt',
  'street': 'Kloaver Blatt',
  'city': 'Velen',
  'county': 'Kreis Borken',
  'state': 'North Rhine-Westphalia',
  'postcode': '46342',
  'country': 'Germany',
  'country_code': 'de',
  'lon': 6.9077083,
  'lat': 51.9163524,
  'formatted': 'Kloaver Blatt, 46342 Velen, Germany',
  'address_line1': 'Kloaver Blatt',
  'address_line2': '46342 Velen, Germany',
  'timezone': {'name': 'Europe/Berlin',
   'offset_STD': '+01:00',
   'offset_STD_seconds': 3600,
   'offset_DST': '+02:00',
   'offset_DST_seconds': 7200,
   'abbreviation_STD': 'CET',
   'abbreviation_DST': 'CEST'},
  'result_type': 'street',
  'rank': {'importance': 0.3,
   'popularity': 4.2339429069255905,
   'confidence': 1,
   'confidence_city_level': 1,
   'confidence_street_level

[GeoPandas](https://github.com/geopandas/geopandas) helps us transform the data into a tabular format. The method parses the `geometry` into a [Shapely](https://github.com/shapely/shapely) geometric object, puts all `properties` into separate columns, and ignores the rest. We also set the coordinate reference system (CRS) to 'EPSG:4326' which means that the tuples in the geometries are interpreted as longitude and latitude.

In [7]:
import geopandas as gpd

df_geocodes = gpd.GeoDataFrame.from_features(results, crs='EPSG:4326')
df_geocodes.iloc[:, :4].head()

/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


,geometry,datasource,name,street
0,POINT (6.90771 51.91635),"{'sourcename': 'openstreetmap', 'attribution':...",Kloaver Blatt,Kloaver Blatt
1,POINT (8.12712 52.26710),"{'sourcename': 'openstreetmap', 'attribution':...",Hasewinkel,Hasewinkel
2,POINT (4.44258 51.18693),"{'sourcename': 'openstreetmap', 'attribution':...",Roderveldlaan,Roderveldlaan
3,POINT (8.98558 52.25417),"{'sourcename': 'openstreetmap', 'attribution':...",Poggenbeeke,Poggenbeeke
4,POINT (3.73198 51.07771),"{'sourcename': 'openstreetmap', 'attribution':...",Galvestonstraat,Galvestonstraat


And we flatten `dict` columns using `pandas.json_normalize`:

In [8]:
df_rank = pd.json_normalize(df_geocodes['rank'])
df_rank.head()

,importance,popularity,confidence,confidence_city_level,confidence_street_level,match_type
0,0.30,4.233943,1.0,1.0,1.0,full_match
1,0.41,5.600543,0.5,1.0,1.0,match_by_street
2,0.41,6.876627,0.5,1.0,1.0,match_by_street
3,0.20,4.911427,1.0,1.0,1.0,full_match
4,0.41,6.493385,0.5,1.0,1.0,match_by_street


## Part 3 - validate location data quality

Geoapify's Geocoding service primarily is for appending coordinates to your address. But it delivers much more, including what you see in the last table `df_rank`. We have published a comprehensive article specifically about data quality [here](https://towardsdatascience.com/deduplicate-and-clean-up-millions-of-location-records-abcffb308ebf).

The `confidence` attributes correlate with quality of the original address inputs. A low score likely means, something went wrong with the corresponding input. That can result in a wrong match, or a match of unexpected type. Below we see what `result_type`s have been matched with our data:

In [9]:
df_geocodes['result_type'].value_counts()

street      883
amenity     159
building     19
postcode      8
suburb        8
city          4
Name: result_type, dtype: int64

Often a match of type `city`, `postcode`, or `suburb` is a result of missing relevant attributes in the original data.. But that's something we could have easily found without geocoding. The more interesting cases are those hidden behind invalid combinations of attributes. Row `374` is such an example. Searching online, it turns out that stadium `KSV Sottegem` is located in city 9620 Zottegem, street Kloosterstraat. Street Moelde as in the original input is nearby and Godveerdegem seems to be a suburb of Zottegem.

We get a quick overview of all likely erroneous cases with the following block:

In [10]:
use_cols = ['street', 'housenumber', 'postcode', 'city', 'country', 'result_type', 'name']
low_confidence = df_rank['confidence'].le(0.25)
df_geocodes_renamed = df_geocodes[use_cols].rename(columns={col: 'geoapify_' + col for col in use_cols})

pd.concat([df, df_rank[['confidence']], df_geocodes_renamed], axis=1).loc[low_confidence]

,Name,Street,Housenumber,Postcode,City,Country,AddressLine1,AddressLine2,confidence,geoapify_street,geoapify_housenumber,geoapify_postcode,geoapify_city,geoapify_country,geoapify_result_type,geoapify_name
56,Alter Sportplatz,Tannenweg,NaN,31855,Selxen,Germany,Tannenweg,31855 Selxen,0.250000,NaN,NaN,31855,Selxen,Germany,postcode,Selxen
61,Bezirkssportanlage Rußheide,B 66,NaN,33607,Bielefeld,Germany,B 66,33607 Bielefeld,0.250000,NaN,NaN,33607,Bielefeld,Germany,suburb,Mitte
143,Sportplatz Bierpohlweg,Bierpohlweg,NaN,32425,Minden,Germany,Bierpohlweg,32425 Minden,0.250000,NaN,NaN,32425,Minden,Germany,suburb,Nordstadt
174,Sportplatz,B 64/83 n,NaN,37671,Godelheim,Germany,B 64/83 n,37671 Godelheim,0.250000,NaN,NaN,NaN,Höxter,Germany,suburb,NaN
374,KSV Sottegem,Moelde,NaN,9620,Godveerdegem,Belgium,Moelde,9620 Godveerdegem,0.250000,NaN,NaN,NaN,Godveerdegem,Belgium,city,NaN
376,Am Gaa,Am Gaa,NaN,5483,Wormeldange-Haut,Luxembourg,Am Gaa,5483 Wormeldange-Haut,0.250000,NaN,NaN,5485,Wormeldange-Haut,Luxembourg,postcode,Wormeldange-Haut
501,SC Sonnborn 07,Sonnborner Ufer,NaN,42327,Wuppertal,Germany,Sonnborner Ufer,42327 Wuppertal,0.250000,NaN,NaN,42327,Wuppertal,Germany,suburb,Gemarkung Vohwinkel
561,Sportplatz Lelbach,B 251,NaN,34497,Lelbach,Germany,B 251,34497 Lelbach,0.158333,Lelbach B,2,34497,Korbach,Germany,building,NaN
593,Sportcentrum Bergheim,A 61,NaN,50126,Thorr,Germany,A 61,50126 Thorr,0.250000,NaN,NaN,50126,Thorr,Germany,postcode,Thorr
771,VFB Schrecksbach,B 254,NaN,34637,Schrecksbach,Germany,B 254,34637 Schrecksbach,0.250000,NaN,NaN,34637,Schrecksbach,Germany,postcode,NaN


## Part 4 - add building details where available

Geo-coordinates are the fundament of geographic data analytics. At the very least, we can start visualizing our addresses on a map as points. But we can do much more.

Below we use geo-coordinates as a reference to extract building details. Geoapify also comes with a handy API for this use case called Place Details. It allows us to extract further attributes for a number of different detail typies. We choose `building`.

Again, our preference is the CLI, using the batch version of the API:

In [11]:
from geobatchpy.batch import parse_geocodes, simplify_batch_place_details_results
from geobatchpy.utils import write_data_to_json_file

geocodes = parse_geocodes(geocodes=[(item.x, item.y) for _, item in df_geocodes['geometry'].items()])

data = {
    'api': '/v2/place-details',  # See the Geoapify API docs for other batch APIs.
    'inputs': geocodes,
    'params': {'features': 'building'},  # can be also several, separated by comma in a single string
    'batch_len': 500,
    'id': 'tutorial-batch-details'
}

write_data_to_json_file(data=data, file_path='tutorial-details-input.json')

We switch again to the CLI to submit jobs, monitor, and retrieve results:

```shell
geobatch submit tutorial-details-input.json tutorial-details-urls.json
```

The output of the first step `tutorial-details-urls.json` is the input for the next:

```shell
geobatch receive tutorial-details-urls.json tutorial-details-results.json
```

Pause here and continue after all your jobs are done. When jobs are done, we parse those again into a simplified list of lists of GeoJSON-like dictionaries.

In [12]:
results = read_data_from_json_file('tutorial-details-results.json')['results']
results = simplify_batch_place_details_results(results)

print('Number of details found per location:')
pd.Series([len(res) for res in results]).value_counts()

Number of details found per location:


0    922
1    159
dtype: int64

In that case, we found `building` details for just 159 records. Most likely, because a lot of the stadiums are just sports fields in the middle of nowhere. Or in optimistic words: the absence of building geometries tells us which sports facilities in our dataset are small and which are not.

And out of the 159 buildings not all carry the propoerty of a `sport.stadium`. Here is one of the positive examples:

In [13]:
results[26]

[{'type': 'Feature',
  'properties': {'feature_type': 'building',
   'categories': ['building', 'building.sport', 'sport', 'sport.stadium'],
   'datasource': {'sourcename': 'openstreetmap',
    'attribution': '© OpenStreetMap contributors',
    'license': 'Open Database Licence',
    'url': 'https://www.openstreetmap.org/copyright',
    'raw': {'name': 'Johan Cruijff ArenA',
     'height': '75 m',
     'osm_id': -1458682,
     'leisure': 'stadium',
     'name:mk': 'Амстердам Арена',
     'ref:bag': 363100012075730,
     'website': 'http://www.amsterdamarena.nl',
     'alt_name': 'Johan Cruijf Arena',
     'building': 'stadium',
     'capacity': 52342,
     'operator': 'Stadion Amsterdam N.V.',
     'osm_type': 'r',
     'wikidata': 'Q207109',
     'addr:city': 'Amsterdam',
     'wikipedia': 'nl:Johan Cruijff ArenA',
     'start_date': 1996,
     'addr:street': 'Arena boulevard',
     'contact:fax': '0203111380',
     'description': 'Johan Cruijff ArenA is a stadium in and has been used

We use GeoPandas to parse the building details into a GeoDataFrame, with index referencing to our original addresses:

In [14]:
index_name = df.index.name if df.index.name is not None else 'index'
df_details = pd.concat([gpd.GeoDataFrame.from_features(res).assign(**{index_name: idx})
                        for idx, res in zip(df.index, results)]).set_index(index_name)
# df_details.geometry = gpd.GeoSeries(df_details.geometry).set_crs('EPSG:4326')
df_details = gpd.GeoDataFrame(df_details, crs='EPSG:4326')
df_details.head()

,geometry,feature_type,categories,datasource,street,city,state,postcode,country,country_code,formatted,address_line1,address_line2,lat,lon,building,area,place_id,housenumber,county,name,operator,facilities,wiki_and_media,website,description,name_other,name_international,contact,opening_hours,heritage,owner,place_of_worship,catering
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,"POLYGON ((8.12688 52.26715, 8.12704 52.26712, ...",building,"[building, building.residential]","{'sourcename': 'openstreetmap', 'attribution':...",Hasewinkel,Osnabrück,Lower Saxony,49086,Germany,de,"Hasewinkel, 49086 Osnabrück, Germany",Hasewinkel,"49086 Osnabrück, Germany",52.267078,8.127102,{'type': 'residential'},79.0,517b8a16b8024120405948a78d5632224a40f00102f901...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,"POLYGON ((4.83247 50.99205, 4.83254 50.99192, ...",building,"[building, building.sport, sport, sport.stadium]","{'sourcename': 'openstreetmap', 'attribution':...",Grote Laakweg,Aarschot,Flemish Brabant,3200,Belgium,be,"Sporting Club Aarschot, Grote Laakweg 29, 3200...",Sporting Club Aarschot,"Grote Laakweg 29, 3200 Aarschot, Belgium",50.992094,4.833120,NaN,1448.0,514e519e332255134059449d122ffd7e4940f00102f901...,29,Leuven,Sporting Club Aarschot,S.C.Aarschot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,"POLYGON ((8.22616 53.14664, 8.22618 53.14653, ...",building,"[building, building.sport, sport, sport.stadiu...","{'sourcename': 'openstreetmap', 'attribution':...",Europaplatz,Oldenburg,Lower Saxony,26123,Germany,de,"kleine EWE Arena, Europaplatz 12, 26123 Oldenb...",kleine EWE Arena,"Europaplatz 12, 26123 Oldenburg, Germany",53.146592,8.226715,"{'levels': 3, 'type': 'sports_centre', 'materi...",4327.0,512fd5b5fc1374204059f2c05778c4924a40f00102f901...,12,NaN,kleine EWE Arena,Weser-Ems-Halle,{'wheelchair': True},"{'wikidata': 'Q879570', 'wikipedia': 'de:EWE A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,"POLYGON ((6.05971 49.61126, 6.05976 49.61124, ...",building,[building],"{'sourcename': 'openstreetmap', 'attribution':...",Rue de Luxembourg,Bertrange,NaN,8077,Luxembourg,lu,"64 Rue de Luxembourg, 8077 Bertrange, Luxembourg",64 Rue de Luxembourg,"8077 Bertrange, Luxembourg",49.611280,6.059800,NaN,152.0,515f1ca3f8403d184059870f29cc3ece4840f00102f901...,64,Canton Luxembourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,"POLYGON ((4.94003 52.31420, 4.94003 52.31420, ...",building,"[building, building.sport, sport, sport.stadium]","{'sourcename': 'openstreetmap', 'attribution':...",Johan Cruijff Boulevard,Amsterdam,North Holland,1101AX,Netherlands,nl,"Johan Cruijff ArenA, Johan Cruijff Boulevard 2...",Johan Cruijff ArenA,"Johan Cruijff Boulevard 29, 1101 AX Amsterdam,...",52.314353,4.942843,"{'type': 'stadium', 'start_date': 1996, 'heigh...",30843.0,515a84df855dc4134059221c66753b284a40f00101f901...,29,NaN,Johan Cruijff ArenA,Stadion Amsterdam N.V.,NaN,"{'wikidata': 'Q207109', 'wikipedia': 'nl:Johan...",http://www.amsterdamarena.nl,Johan Cruijff ArenA is a stadium in and has be...,{'alt_name': 'Johan Cruijf Arena'},{'mk': 'Амстердам Арена'},"{'phone': '0203111333', 'email': 'info@amsterd...",NaN,NaN,NaN,NaN,NaN


## Part 5 - quick intro into GeoPandas data manipulation

We cover two common GeoPandas data manipulation capabilities:

1. Component-wise summary statistics for geometries.
2. Geospatial joins.

Remember that `df_details` contains geometries about buildings. One such example is given below for the `Johan Cruijff Arena` which is line 26 in `df_details`:

![Shape of Johan Cruijff Arena](shape-stadium.png)

Now with the help of GeoPandas we can easily compute the occupied area and total length of boundaries. We only need to make sure to use a CRS that makes sense for our data. Here we pick 'EPSG:3035' which works well for European geometries and translates to the metric system:

In [15]:
(pd.concat([df_details.geometry.to_crs(crs='EPSG:3035').area,
            df_details.geometry.boundary.to_crs(crs='EPSG:3035').length], axis=1)
 .rename(columns={0: 'Area', 1: 'BoundaryLength'}).head())

,Area,BoundaryLength
index,,
1,79.595084,36.810346
10,1449.534940,213.284523
17,4335.556706,234.403371
21,152.444623,66.759744
26,30895.358101,1366.622927


We illustrate geospatial joins with the help of a 2nd dataset of airports from [opentraveldata](https://github.com/opentraveldata/opentraveldata/tree/master/data/ourairports),  under [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/). The data contains more than just geocoordinates and airport names but we keep it simple for now:

In [16]:
from shapely.geometry import Point

df_airports = pd.read_csv('airports.csv')
df_airports = (df_airports.loc[df_airports.type.isin(['medium_airport', 'large_airport'])]
               .rename(columns={'name': 'AirportName'}).reset_index(drop=True))

geometry = [Point(xy) for xy in zip(df_airports.longitude_deg, df_airports.latitude_deg)]
df_airports = gpd.GeoDataFrame(df_airports, geometry=geometry, crs='EPSG:4326')[['geometry', 'AirportName']]
df_airports.head()

,geometry,AirportName
0,POINT (-158.61800 59.28260),Aleknagik / New Airport
1,POINT (69.80734 33.28460),Khost International Airport
2,POINT (160.05499 -9.42800),Honiara International Airport
3,POINT (157.26300 -8.32797),Munda Airport
4,POINT (102.35224 32.53154),Hongyuan Airport


A geospatial join based on proximity answers us which is the closest airport for every address in our original data, and how far away is that airport (bird distance). Again, we use a CRS working well for European geometries and translating into the metric system:

In [17]:
df_join = (df_geocodes.to_crs('EPSG:3035')
           .sjoin_nearest(df_airports.to_crs('EPSG:3035'),
                          how='left', distance_col='DistanceToAirport')
           .to_crs('EPSG:4326').rename(columns={'AirportName': 'ClosestAirport'})
           .drop('index_right', axis=1))

show_cols = ['formatted', 'ClosestAirport', 'DistanceToAirport']
df_join[show_cols]

,formatted,ClosestAirport,DistanceToAirport
0,"Kloaver Blatt, 46342 Velen, Germany",Twente Airport,40024.716024
1,"Hasewinkel, 49086 Osnabrück, Germany",Münster Osnabrück Airport,33641.511932
2,"Roderveldlaan, 2600 Antwerp, Belgium",Antwerp International Airport (Deurne),1267.311428
3,"Poggenbeeke, 32457 Porta Westfalica, Germany",Bückeburg Air Base,7128.372785
4,"Galvestonstraat, 9000 Ghent, Belgium",Wevelgem Airport,47020.704761
...,...,...,...
1076,"Mühltalweg, 67551 Worms, Germany",Mannheim-City Airport,23035.716161
1077,"Omer Vanaudenhovelaan, 3290 Diest, Belgium",Beauvechain Air Base,32774.379561
1078,"Uferstraße, 35576 Wetzlar, Germany",Siegerland Airport,33732.677575
1079,"Am Pfad, 52525 Braunsrath, Germany",Geilenkirchen Air Base,10500.661219
